# Compara Planilhas Gestores 

In [ ]:
# 1. Importar as bibliotecas necessárias
import pandas as pd
import os

print("Bibliotecas importadas com sucesso.")

# 2. Definir os caminhos dos arquivos
input_dir = r"C:\DIRETORIO\INPUT"
output_dir = r"C:\DIRETORIO\INPUT"

# Nomes dos arquivos
file_old_name = "ARQUIVO_ENTRADA_1.xlsx" 
file_new_name = "ARQUIVO_ENTRADA_2.xlsx"

# ---- ALTERAÇÃO 1 AQUI ----
# Mude a extensão de .xls para .xlsx
file_output_name = "ARQUIVO_SAIDA.xlsx" 
# -------------------------

# Criar os caminhos completos
path_old = os.path.join(input_dir, file_old_name)
path_new = os.path.join(input_dir, file_new_name)
path_output = os.path.join(output_dir, file_output_name)

# Garantir que o diretório de saída exista
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Diretório de saída criado em: {output_dir}")

# 3. Ler as planilhas
try:
    print(f"Lendo planilha desatualizada: {file_old_name}...")
    df_old = pd.read_excel(path_old)
    
    print(f"Lendo planilha atualizada: {file_new_name}...")
    df_new = pd.read_excel(path_new) 
    
    print("Leitura dos arquivos concluída.")

except FileNotFoundError as e:
    print(f"Erro: Arquivo não encontrado. Verifique o caminho. {e}")
except Exception as e:
    print(f"Ocorreu um erro ao ler os arquivos: {e}")


# 4. Lógica de Atualização
if 'df_old' in locals() and 'df_new' in locals():
    print("Iniciando processo de atualização dos dados...")

    # ---- ALTERAÇÃO 3 AQUI ----
    # Garante que a chave composta ('NO_MUNICIPIO', 'SG_UF') seja única
    print(f"Linhas em df_old (antes da limpeza): {len(df_old)}")
    df_old = df_old.drop_duplicates(subset=["NO_MUNICIPIO", "SG_UF"], keep='first')
    print(f"Linhas em df_old (após limpeza): {len(df_old)}")

    # ---- ALTERAÇÃO 4 AQUI ----
    print(f"Linhas em df_new (antes da limpeza): {len(df_new)}")
    df_new = df_new.drop_duplicates(subset=["NO_MUNICIPIO", "SG_UF"], keep='first')
    print(f"Linhas em df_new (após limpeza): {len(df_new)}")

    # ---- ALTERAÇÃO 5 AQUI ----
    # Define as colunas 'NO_MUNICIPIO' e 'SG_UF' como o índice (MultiIndex)
    print("Definindo índices compostos...")
    df_old_indexed = df_old.set_index(["NO_MUNICIPIO", "SG_UF"])
    df_new_indexed = df_new.set_index(["NO_MUNICIPIO", "SG_UF"])
    # -------------------------

    cols_to_update = ["NO_PESSOA", "EMAIL", "TELEFONE"]
    df_new_updates = df_new_indexed[cols_to_update]

    print("Executando a atualização (update)...")
    # O Update vai funcionar corretamente com o MultiIndex
    df_old_indexed.update(df_new_updates)
    print("Atualização concluída.")

    # O reset_index vai trazer 'NO_MUNICIPIO' e 'SG_UF' de volta como colunas
    df_updated = df_old_indexed.reset_index()

    # 5. Remover Duplicatas Completas
    print("Removendo duplicatas completas (linhas 100% idênticas)...")
    records_before = len(df_updated)
    df_final = df_updated.drop_duplicates()
    records_after = len(df_final)
    print(f"Foram removidas {records_before - records_after} linhas duplicadas.")

    # 6. Salvar o resultado
    try:
        print(f"Salvando arquivo final em: {path_output}...")
        
        # ---- ALTERAÇÃO 2 AQUI ----
        # Remova o engine='xlwt'. O Pandas usará o 'openpyxl' automaticamente.
        df_final.to_excel(path_output, index=False)
        # -------------------------
        
        print("\n--- SUCESSO ---")
        print(f"Arquivo '{file_output_name}' foi salvo com sucesso!")

    except Exception as e:
        print(f"Ocorreu um erro ao salvar o arquivo: {e}")